##Dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as df

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torch
import transformers

##Model Prep

In [ ]:
# Define the categories
categories = ["very negative", "negative", "neutral", "positive", "very positive"]
label_to_score = {"very negative": -2, "negative": -1, "neutral": 0, "positive": 1, "very positive": 2}
gpt_scores = {1: "very negative", 2: "negative", 3: "neutral", 4: "positive", 5: "very positive"}
# llama_cat_string = flan_cat_string = str(categories)
llama_cat_string = flan_cat_string = ', '.join(categories)
gpt_cat_string = str(gpt_scores)
check_categories = set(categories)
print(gpt_cat_string)
print(flan_cat_string)

{1: 'very negative', 2: 'negative', 3: 'neutral', 4: 'positive', 5: 'very positive'}
very negative, negative, neutral, positive, very positive


#Dataset Loading

##Test Set

Test Sentences


In [ ]:
# Input sentence
#sentences = ["This opinion is neither good nor bad", "I have extreme hatred for this", "I am not a fan of this", "This was the BEST experience", "It was okay!", "I didn't hate it"]
sentences = ["I have extreme hatred for this", "I am not a fan of this", "It was alright", "This was a wonderful experience!", "I enjoyed it."]



In [ ]:
for sent in sentences:
  # print(gpt_infer(sent, gpt_cat_string))
  print(gpt_infer(sent, gpt_cat_string))

-2
-1
0
2
1


##Actual Datasets

Idiom Data

In [ ]:
idioms = idiom_sent_score = pd.read_csv("/content/drive/MyDrive/FigSemData/figsem_paired_data/idata_raw.tsv", sep = '\t', header = 0)
idiom_sent_score = idiom_sent_score.reindex(columns=idiom_sent_score.columns.tolist()+["idiom-label", "literal-label"])
idiom_sent_score.head(250)

,idiom,literal,idiom-label,literal-label
0,"For example , evidence of the implications of ...","For example , evidence of the implications of ...",NaN,NaN
1,I remember when I was about two - and - a - ha...,I remember when I was about two - and - a - ha...,NaN,NaN
2,The result is that ordinary motorists can now ...,The result is that ordinary motorists can now ...,NaN,NaN
3,Cynics say — cynics have already said — when ...,Cynics say — cynics have already said — when ...,NaN,NaN
4,A 37-year - old woman accused of mortgage frau...,A 37-year - old woman accused of mortgage frau...,NaN,NaN
...,...,...,...,...
245,"Stephen , you 'll have to have words with Grah...","Stephen , you 'll have to have a serious discu...",NaN,NaN
246,Daley Thompson is launching a new career in th...,Daley Thompson is launching a new career in an...,NaN,NaN
247,But an intimate dinner at the château was anot...,But an intimate dinner at the château was anot...,NaN,NaN
248,"There 's definitely that needed and , and by a...","There 's definitely that needed and , and ever...",NaN,NaN


In [ ]:
idiom_sent_score

,idiom,literal,idiom-label,literal-label
0,"For example , evidence of the implications of ...","For example , evidence of the implications of ...",NaN,NaN
1,I remember when I was about two - and - a - ha...,I remember when I was about two - and - a - ha...,NaN,NaN
2,The result is that ordinary motorists can now ...,The result is that ordinary motorists can now ...,NaN,NaN
3,Cynics say — cynics have already said — when ...,Cynics say — cynics have already said — when ...,NaN,NaN
4,A 37-year - old woman accused of mortgage frau...,A 37-year - old woman accused of mortgage frau...,NaN,NaN
...,...,...,...,...
246,Daley Thompson is launching a new career in th...,Daley Thompson is launching a new career in an...,NaN,NaN
247,But an intimate dinner at the château was anot...,But an intimate dinner at the château was anot...,NaN,NaN
248,"There 's definitely that needed and , and by a...","There 's definitely that needed and , and ever...",NaN,NaN
249,We know employees are being blackmailed with t...,We know employees are being blackmailed with t...,NaN,NaN


Metaphor Data

In [ ]:
mphors = mphor_sent_score = pd.read_csv("/content/drive/MyDrive/FigSemData/figsem_paired_data/mdata_raw.tsv", sep = '\t', header = 0)
mphor_sent_score = mphor_sent_score.reindex(columns=mphor_sent_score.columns.tolist()+["metaphor-label", "literal-label"])

In [ ]:
mphor_sent_score

,metaphor,literal,metaphor-label,literal-label
0,he absorbed the costs for the accident.,he paid for the costs for the accident .,NaN,NaN
1,the path climbed all the way to the top of the...,the path ascended all the way to the top of th...,NaN,NaN
2,some nice results emerged from the study,some nice results happened as a result of the ...,NaN,NaN
3,her feelings condensed,her feelings became more compact,NaN,NaN
4,our new synthetic fabric breathes and is perfe...,our new synthetic fabric exchanges gases with ...,NaN,NaN
...,...,...,...,...
195,she had to juggle her job and her children,she had to manage her job and her children at ...,NaN,NaN
196,"Guskewau , the darkness , wrapped her","Guskewau , the darkness , surrounded her",NaN,NaN
197,don't bolt your food!,don't swallow your food without chewing !,NaN,NaN
198,the businessman kited millions of dollars,the businessman got millions of dollars by usi...,NaN,NaN


Hyperbole Data

In [ ]:
hypos = hypo_sent_score = pd.read_csv("/content/drive/MyDrive/FigSemData/figsem_paired_data/hdata_raw.tsv", sep = '\t', header = 0)
hypo_sent_score = hypo_sent_score.reindex(columns=hypo_sent_score.columns.tolist()+["metaphor-label", "literal-label"])

In [ ]:
hypo_sent_score

,hyperbole,literal,metaphor-label,literal-label
0,"I'm a burst of colors today, I'm wearing a yel...","I'm very colorful today, I'm wearing a yellow ...",NaN,NaN
1,I write my diary when I feel crushed by the we...,I write my diary when I feel under pressure.,NaN,NaN
2,The town I grew up in is so isolated that rock...,The town I grew up in was isolated and behind ...,NaN,NaN
3,The Eagle soars in the summit of the azure.,The Eagle soars high in the sky.,NaN,NaN
4,You should sunbathe! You look white as a ghost!,You should sunbathe! You look pale!,NaN,NaN
...,...,...,...,...
245,"His classmates laughed at him, saying he had a...",His classmates laughed at him saying he had a ...,NaN,NaN
246,I am in the middle of an emotional tornado.,I am in the middle of emotional turmoil.,NaN,NaN
247,"If Sylvia flirted with Johnny, Steve threatene...","If Sylvia flirted with Johnny, Steve threatene...",NaN,NaN
248,Christmas will never come.,Christmas will take a long time to come.,NaN,NaN


#Running Sentiment Analysis...

In [ ]:
def run_sent_analysis(in_df, out_df, infer=gpt_infer, cats=gpt_cat_string):
  for idx, row in in_df.iterrows():
    fig_sent = row[in_df.keys()[0]]
    lit_sent = row[in_df.keys()[1]]
    fig_label = infer(fig_sent, cats)
    lit_label = infer(lit_sent, cats)
    if fig_label is None or lit_label is None:
      print("UH-OH!")
    out_df.at[idx, out_df.keys()[2]] = fig_label
    out_df.at[idx, out_df.keys()[3]] = lit_label
  return out_df

In [ ]:
run_sent_analysis(idioms, idiom_sent_score)

,idiom,literal,idiom-label,literal-label
0,"For example , evidence of the implications of ...","For example , evidence of the implications of ...",0.0,0.0
1,I remember when I was about two - and - a - ha...,I remember when I was about two - and - a - ha...,0.0,0.0
2,The result is that ordinary motorists can now ...,The result is that ordinary motorists can now ...,1.0,1.0
3,Cynics say — cynics have already said — when ...,Cynics say — cynics have already said — when ...,-1.0,-1.0
4,A 37-year - old woman accused of mortgage frau...,A 37-year - old woman accused of mortgage frau...,0.0,-1.0
...,...,...,...,...
246,Daley Thompson is launching a new career in th...,Daley Thompson is launching a new career in an...,1.0,1.0
247,But an intimate dinner at the château was anot...,But an intimate dinner at the château was anot...,-1.0,-1.0
248,"There 's definitely that needed and , and by a...","There 's definitely that needed and , and ever...",0.0,0.0
249,We know employees are being blackmailed with t...,We know employees are being blackmailed with t...,-2.0,-2.0


In [ ]:
idiom_sent_score

,idiom,literal,idiom-label,literal-label
0,"For example , evidence of the implications of ...","For example , evidence of the implications of ...",0.0,0.0
1,I remember when I was about two - and - a - ha...,I remember when I was about two - and - a - ha...,0.0,0.0
2,The result is that ordinary motorists can now ...,The result is that ordinary motorists can now ...,1.0,1.0
3,Cynics say — cynics have already said — when ...,Cynics say — cynics have already said — when ...,-1.0,-1.0
4,A 37-year - old woman accused of mortgage frau...,A 37-year - old woman accused of mortgage frau...,0.0,-1.0
...,...,...,...,...
246,Daley Thompson is launching a new career in th...,Daley Thompson is launching a new career in an...,1.0,1.0
247,But an intimate dinner at the château was anot...,But an intimate dinner at the château was anot...,-1.0,-1.0
248,"There 's definitely that needed and , and by a...","There 's definitely that needed and , and ever...",0.0,0.0
249,We know employees are being blackmailed with t...,We know employees are being blackmailed with t...,-2.0,-2.0


In [ ]:
from operator import indexOf
def compute_quant_values(out_df, dset="idiom"):
  out_df = out_df.assign(valence = lambda x: (x[out_df.keys()[2]] - x[out_df.keys()[3]]))
  out_df["bin"] = out_df["valence"].apply(lambda x: 0 if abs(x)>1 else 1)
  return out_df

In [ ]:
idiom_sent_score = compute_quant_values(idiom_sent_score)

In [ ]:
idiom_sent_score

,idiom,literal,idiom-label,literal-label,valence,bin
0,"For example , evidence of the implications of ...","For example , evidence of the implications of ...",0.0,0.0,0.0,1
1,I remember when I was about two - and - a - ha...,I remember when I was about two - and - a - ha...,0.0,0.0,0.0,1
2,The result is that ordinary motorists can now ...,The result is that ordinary motorists can now ...,1.0,1.0,0.0,1
3,Cynics say — cynics have already said — when ...,Cynics say — cynics have already said — when ...,-1.0,-1.0,0.0,1
4,A 37-year - old woman accused of mortgage frau...,A 37-year - old woman accused of mortgage frau...,0.0,-1.0,1.0,1
...,...,...,...,...,...,...
246,Daley Thompson is launching a new career in th...,Daley Thompson is launching a new career in an...,1.0,1.0,0.0,1
247,But an intimate dinner at the château was anot...,But an intimate dinner at the château was anot...,-1.0,-1.0,0.0,1
248,"There 's definitely that needed and , and by a...","There 's definitely that needed and , and ever...",0.0,0.0,0.0,1
249,We know employees are being blackmailed with t...,We know employees are being blackmailed with t...,-2.0,-2.0,0.0,1


In [ ]:
idiom_sent_score.to_csv('/content/drive/MyDrive/FigSemData/figsem_inference_data/idata_gpt.csv')

In [ ]:
mphors

,metaphor,literal
0,he absorbed the costs for the accident.,he paid for the costs for the accident .
1,the path climbed all the way to the top of the...,the path ascended all the way to the top of th...
2,some nice results emerged from the study,some nice results happened as a result of the ...
3,her feelings condensed,her feelings became more compact
4,our new synthetic fabric breathes and is perfe...,our new synthetic fabric exchanges gases with ...
...,...,...
195,she had to juggle her job and her children,she had to manage her job and her children at ...
196,"Guskewau , the darkness , wrapped her","Guskewau , the darkness , surrounded her"
197,don't bolt your food!,don't swallow your food without chewing !
198,the businessman kited millions of dollars,the businessman got millions of dollars by usi...


In [ ]:
mphor_sent_score = run_sent_analysis(mphors, mphor_sent_score)

In [ ]:
mphor_sent_score = compute_quant_values(mphor_sent_score)

In [ ]:
mphor_sent_score

,metaphor,literal,metaphor-label,literal-label,valence,bin
0,he absorbed the costs for the accident.,he paid for the costs for the accident .,0.0,0.0,0.0,1
1,the path climbed all the way to the top of the...,the path ascended all the way to the top of th...,0.0,1.0,-1.0,1
2,some nice results emerged from the study,some nice results happened as a result of the ...,1.0,1.0,0.0,1
3,her feelings condensed,her feelings became more compact,0.0,0.0,0.0,1
4,our new synthetic fabric breathes and is perfe...,our new synthetic fabric exchanges gases with ...,2.0,1.0,1.0,1
...,...,...,...,...,...,...
195,she had to juggle her job and her children,she had to manage her job and her children at ...,0.0,0.0,0.0,1
196,"Guskewau , the darkness , wrapped her","Guskewau , the darkness , surrounded her",0.0,-1.0,1.0,1
197,don't bolt your food!,don't swallow your food without chewing !,0.0,0.0,0.0,1
198,the businessman kited millions of dollars,the businessman got millions of dollars by usi...,-1.0,-1.0,0.0,1


In [ ]:
mphor_sent_score.to_csv('/content/drive/MyDrive/FigSemData/figsem_inference_data/mdata_gpt.csv')

In [ ]:
hypo_sent_score

,hyperbole,literal,metaphor-label,literal-label,valence,bin
0,"I'm a burst of colors today, I'm wearing a yel...","I'm very colorful today, I'm wearing a yellow ...",1.0,0.0,1.0,1
1,I write my diary when I feel crushed by the we...,I write my diary when I feel under pressure.,-1.0,0.0,-1.0,1
2,The town I grew up in is so isolated that rock...,The town I grew up in was isolated and behind ...,-1.0,-1.0,0.0,1
3,The Eagle soars in the summit of the azure.,The Eagle soars high in the sky.,1.0,1.0,0.0,1
4,You should sunbathe! You look white as a ghost!,You should sunbathe! You look pale!,0.0,1.0,-1.0,1
...,...,...,...,...,...,...
245,"His classmates laughed at him, saying he had a...",His classmates laughed at him saying he had a ...,-1.0,-1.0,0.0,1
246,I am in the middle of an emotional tornado.,I am in the middle of emotional turmoil.,-1.0,-1.0,0.0,1
247,"If Sylvia flirted with Johnny, Steve threatene...","If Sylvia flirted with Johnny, Steve threatene...",-1.0,-1.0,0.0,1
248,Christmas will never come.,Christmas will take a long time to come.,-1.0,0.0,-1.0,1


In [ ]:
hypo_sent_score = run_sent_analysis(hypos, hypo_sent_score)

In [ ]:
hypo_sent_score

,hyperbole,literal,metaphor-label,literal-label,valence,bin
0,"I'm a burst of colors today, I'm wearing a yel...","I'm very colorful today, I'm wearing a yellow ...",1.0,1.0,1.0,1
1,I write my diary when I feel crushed by the we...,I write my diary when I feel under pressure.,-1.0,0.0,-1.0,1
2,The town I grew up in is so isolated that rock...,The town I grew up in was isolated and behind ...,0.0,-1.0,0.0,1
3,The Eagle soars in the summit of the azure.,The Eagle soars high in the sky.,1.0,0.0,0.0,1
4,You should sunbathe! You look white as a ghost!,You should sunbathe! You look pale!,0.0,1.0,-1.0,1
...,...,...,...,...,...,...
245,"His classmates laughed at him, saying he had a...",His classmates laughed at him saying he had a ...,-1.0,-1.0,0.0,1
246,I am in the middle of an emotional tornado.,I am in the middle of emotional turmoil.,-1.0,-1.0,0.0,1
247,"If Sylvia flirted with Johnny, Steve threatene...","If Sylvia flirted with Johnny, Steve threatene...",-1.0,-1.0,0.0,1
248,Christmas will never come.,Christmas will take a long time to come.,-1.0,0.0,-1.0,1


In [ ]:
hypo_sent_score = compute_quant_values(hypo_sent_score)

In [ ]:
hypo_sent_score

,hyperbole,literal,metaphor-label,literal-label,valence,bin
0,"I'm a burst of colors today, I'm wearing a yel...","I'm very colorful today, I'm wearing a yellow ...",1.0,1.0,0.0,1
1,I write my diary when I feel crushed by the we...,I write my diary when I feel under pressure.,-1.0,0.0,-1.0,1
2,The town I grew up in is so isolated that rock...,The town I grew up in was isolated and behind ...,0.0,-1.0,1.0,1
3,The Eagle soars in the summit of the azure.,The Eagle soars high in the sky.,1.0,0.0,1.0,1
4,You should sunbathe! You look white as a ghost!,You should sunbathe! You look pale!,0.0,1.0,-1.0,1
...,...,...,...,...,...,...
245,"His classmates laughed at him, saying he had a...",His classmates laughed at him saying he had a ...,-1.0,-1.0,0.0,1
246,I am in the middle of an emotional tornado.,I am in the middle of emotional turmoil.,-1.0,-1.0,0.0,1
247,"If Sylvia flirted with Johnny, Steve threatene...","If Sylvia flirted with Johnny, Steve threatene...",-1.0,-1.0,0.0,1
248,Christmas will never come.,Christmas will take a long time to come.,-1.0,0.0,-1.0,1


In [ ]:
hypo_sent_score.to_csv('/content/drive/MyDrive/FigSemData/figsem_inference_data/hypo_gpt.csv')

#Models


##FLAN-T5


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the FLAN-T5 base model and tokenizer
model_name = "google/flan-t5-xl"
tokenizer_flan = T5Tokenizer.from_pretrained(model_name)
model_flan = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def flanT5_infer(sentence, cat_string, model=model_flan, tokenizer=tokenizer_flan, score_convert=label_to_score):
    # Create the prompt with categories
    prompt = f"Please perform Sentiment Classification task. Given the sentence below, assign a sentiment score that corresponds to one of the following categories: ({cat_string}). Return only the score value without any other text. \nText: '{sentence}' \nSentiment:"
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    outputs = model.generate(input_ids, temperature=1)

    # Decode the output
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    label = result.strip().lower()
    score = label_to_score[label] if label in label_to_score.keys() else None
    return score

##LLaMa-3

### Hugging Face Stuff

In [ ]:
!huggingface-cli download meta-llama/Meta-Llama-3-8B-Instruct --include "original/*" --local-dir Meta-Llama-3-8B-Instruct

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_errors.py", line 286, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.10/dist-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models/meta-llama/Meta-Llama-3-8B-Instruct/revision/main

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    service.run()
  File "/usr/local/lib

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


###Pipeline

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
  "text-generation",
  model="meta-llama/Meta-Llama-3-8B-Instruct",
  model_kwargs={"torch_dtype": torch.bfloat16},
  device="cuda",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# The meta/meta-llama-3-70b-instruct model can stream output as it's running.

def llama_infer(sentence, cat_string, label_to_score = label_to_score, pipeline = pipeline):
  prompt = f"Please perform Sentiment Classification task. Given the sentence below, assign a sentiment score that corresponds to one of the following categories {cat_string}. Return only the score value without any other text. \nText: '{sentence}' \nSentiment:"
  messages = [
      {"role": "user", "content": prompt},
  ]

  input = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
    input,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1,
    top_p=0.9,
    return_full_text=False
    )
  # return ""
  label = outputs[0]["generated_text"].strip().lower()
  score = label_to_score[label] if label in label_to_score.keys() else None
  return score





In [ ]:
outputs = {}
for sent in sentences:
  outputs[sent] = llama_infer(sent)
  # print(llama_infer(sent))

print(outputs)

##LLaMa 2

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM

# Load the Llama 2 model and tokenizer
model_name = "meta-llama/Llama-2-7b"  # Specify the model variant
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

for sentence in sentences:
  # Create the prompt with categories
  prompt = f"Please perform Sentiment Classification task. Given the sentence below, assign a sentiment score that corresponds to one of the following categories ({categories}). Return only the score value. \nText: '{sentence}' \nSentiment:"

  # Tokenize the input
  input_ids = tokenizer(prompt, return_tensors='pt').input_ids

  # Generate output with limited token length
  outputs = model.generate(
      input_ids,
      max_length=20,  # Limit the token generation length
      temperature=1,  # Controls randomness
      early_stopping=True  # Stop early to avoid excessive output
  )

  # Decode the output
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print("Predicted sentiment:", result)



OSError: Can't load tokenizer for 'meta-llama/Llama-2-7b'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Llama-2-7b' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.

##GPT-4

In [ ]:
# Install the OpenAI package if you haven't already
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


In [ ]:
def gpt_infer(sentence, cat_string):
  import os
  from openai import OpenAI
  client = OpenAI(api_key = "") #use your OAI key from a securely stored location
  # Create the prompt with categories
  prompt = f"Please perform Sentiment Classification task. Given the sentence below, assign a sentiment score that corresponds to one of the following categories {cat_string}. Return only the score value without any other text. \nText: '{sentence}' \nSentiment:"

    # Generate response with a limited number of tokens
  response = client.chat.completions.create(
      model="gpt-4",
      messages=[{"role": "user", "content": prompt}],
      temperature=1  # Controls randomness (higher values = more random)
  )

    # Extract the generated response
  sentiment = response.choices[0].message.content
  return int(sentiment.strip().lower()) - 3 #shifting the rating to a [-2, 2] scale